# Ingeniería de características

Grupo: 1

Estudiantes:

-Constanza Olivos Fernandez

-Javier Nanco Becerra

-Nicolás Pozo Villagrán

Fecha: 21-09-2025

Version: 1.0

## 📑 README – Features Derivadas – Dataset Sintético de Trigo (La Araucanía)

### 1. Descripción
Este conjunto de **features derivadas** fue creado a partir del dataset sintético de trigo para mejorar la predicción del rendimiento en **invierno y primavera**.  

Las variables se construyeron a partir de condiciones climáticas, nutricionales y de manejo agrícola, con el fin de capturar **patrones adicionales de crecimiento, fertilidad y estrés** que no están explícitos en las variables originales.  

---

### 2. Features derivadas

| **Nombre**            | **Tipo**    | **Descripción**                                   | **Cómo se calcula** | **Justificación / Notas** |
|------------------------|-------------|---------------------------------------------------|---------------------|---------------------------|
| **GDD_norm**           | Numérica    | Grados-día normalizados por radiación solar       | `GDD / Horas_luz`   | Relaciona calor útil con horas de luz, midiendo la eficiencia térmica de cada temporada. |
| **Indice_fertilidad**  | Numérica    | Índice ponderado de nutrientes (N, P, K)          | `0.5*N + 0.3*P + 0.2*K` | Resume la fertilización en una métrica única. Los pesos reflejan mayor importancia del Nitrógeno. |
| **Clima_reglas**       | Categórica  | Clasificación climática por reglas simples        | Si `Lluvia > 600` y `Temp_media < 18` → `húmedo_frío`; si `Lluvia < 300` y `Temp_media > 20` → `seco_cálido`; sino → `moderado` | Identifica condiciones extremas que afectan el cultivo. |
| **Rango_termico_cat**  | Categórica  | Clasificación del rango térmico                   | `<6` = `estable`; `6–10` = `moderado`; `>10` = `variable` | Permite distinguir temporadas con alta o baja amplitud térmica. |
| **Labranza_binary**    | Binaria     | Tipo de labranza codificado                       | `1` si convencional, `0` si cero labranza | Facilita el uso de labranza en modelos de ML. |
| **Plagas_binary**      | Binaria     | Presencia de plagas                               | `1` si “sí”, `0` si “no” | Convierte un factor cualitativo en cuantitativo. |
| **Deficiencia_binary** | Binaria     | Deficiencia nutricional                           | `1` si “sí”, `0` si “no` | Facilita medir impacto de deficiencias. |
| **Problema**           | Numérica    | Total de problemas agrícolas                      | `Plagas_binary + Deficiencia_binary` | Resume en un solo indicador el efecto de plagas y deficiencias. |

---

### 3. Notas generales
- Las **features derivadas** enriquecen el dataset y permiten que los modelos de machine learning detecten patrones no lineales.  
- Dado que los datos son solo de **invierno y primavera**, no se incluyeron acumulados de lluvia.  
- Se recomienda usar las variables originales **junto con estas derivadas** para el entrenamiento.  
- Las variables categóricas (`Clima_reglas`, `Rango_termico_cat`) pueden transformarse a **one-hot encoding** si el modelo lo requiere.  


In [2]:
import pandas as pd
import numpy as np
import os

# ========================
# Cargar CSV original desde la carpeta 'csv'
# ========================
csv_path = os.path.join('csv', 'Dataset_Demostracion.csv')
df = pd.read_csv(csv_path, sep=',')

# ========================
# VARIABLES DERIVADAS BÁSICAS
# ========================

# 1. Grados-día de crecimiento normalizado por horas de luz
df['GDD_norm'] = df['Temp_media'].apply(lambda x: max(x - 10, 0)) / df['Horas_luz']

# 2. Fertilidad total ponderada
df['Indice_fertilidad'] = 0.5*df['N_kg_ha'] + 0.3*df['P_kg_ha'] + 0.2*df['K_kg_ha']

# 3. Categoría de clima según reglas simples
def clasificar_clima(row):
    if row['Lluvia_mm'] > 600 and row['Temp_media'] < 18:
        return 'húmedo_frío'
    elif row['Lluvia_mm'] < 300 and row['Temp_media'] > 20:
        return 'seco_calido'
    else:
        return 'moderado'
df['Clima_reglas'] = df.apply(clasificar_clima, axis=1)

# 4. Rango térmico
df['Rango_termico'] = df['Temp_max'] - df['Temp_min']

# 5. Rango térmico categorizado
def categorizar_rango(rango):
    if rango < 6:
        return 'estable'
    elif rango <= 10:
        return 'moderado'
    else:
        return 'variable'
df['Rango_termico_cat'] = df['Rango_termico'].apply(categorizar_rango)

# ========================
# VARIABLES BINARIAS
# ========================
df['Labranza_binary'] = df['Labranza'].apply(lambda x: 1 if x=='convencional' else 0)
df['Plagas_binary'] = df['Plagas'].apply(lambda x: 1 if x=='sí' else 0)
df['Deficiencia_binary'] = df['Deficiencia'].apply(lambda x: 1 if x=='sí' else 0)
df['Problema'] = df['Plagas_binary'] + df['Deficiencia_binary']

# ========================
# Guardar CSV final con todas las features derivadas en la misma carpeta 'csv'
# ========================
output_path = os.path.join('csv', 'trigo.csv')
df.to_csv(output_path, index=False)
print("✅ CSV con features derivadas creado y listo para modelos de ML.")


FileNotFoundError: [Errno 2] No such file or directory: 'csv\\Dataset_Demostracion.csv'